# 全国 地铁/国铁 线路 GIS 数据分析
本Notebook用于从OpenStreetMap GIS数据中提取、分析并可视化上海地铁11/17号线、广州地铁18号线、北京大兴机场线、西安西户线等多条线路的GIS数据

## 数据处理流程
1. 遍历目录中的多个Shapefile文件，寻找包含某车站("上海汽车城")的记录
2. 将找到的信息与所提供的在线坐标进行比较
3. 计算离线地图中的位置与在线信息的距离
4. 创建可视化地图，同时显示:
   - 上海地铁/广州地铁网络
   - 在线坐标点
   - 从Shapefile中找到的点或要素
   - 距离信息和比例尺
5. 不同颜色标注11号线中的桥梁(bridge)、隧道(tunnel)和地面段
   -  桥梁段（bridge='T'）：使用红色表示，标签为"高架桥段"
   -  隧道段（tunnel='T'）：使用绿色表示，标签为"地下隧道段"
   -  普通路段：使用蓝色表示，标签为"地面段"

## 设置环境
   - pip install geopandas matplotlib pandas ipykernel ipympl contextily
   - 数据源: https://www.openstreetmap.org/ 与 https://download.geofabrik.de/
   - 文件名称：shanghai-latest-free.shp.zip。解压即可得到一套 ESRI Shapefile（通常包括 .shp, .dbf, .shx 等文件），其中按要素类型拆分成多个图层，如道路、铁路、建筑面等。可在 ArcGIS Pro 软件里快速加载上海的路网、铁路并获取其坐标信息，或用 GeoPandas 进行常规矢量数据处理

## OSM数据文件的区别分析
   1. gis_osm_transport_a_free_1.shp - 侧重交通运输基础设施；有无"_a"后缀区分面状和点/线状数据
   - 包含面状交通设施数据（"_a"表示area）
   - 存储车站、港口、机场等有明确边界的交通设施区域
   - 包含车站建筑物的实际几何形状和面积
   - 用于可视化站点区域及其与周围环境的关系
  
  2. gis_osm_traffic_free_1.shp - 侧重交通管理设施
   - 包含交通管理的点状数据
   - 存储交通信号灯、路标、交通标志等点状设施
   - 不包含面积信息，仅有位置坐标

   3. gis_osm_traffic_a_free_1.shp
   - 包含交通管理的面状数据
   - 存储停车场区域、交通管制区、收费站区域等
   - 与transport不同，traffic侧重于管理相关的交通设施

   4. gis_osm_roads_free_1.shp
   - 包含道路网络的线状数据
   - 存储各级道路、街道、高速公路等
   - 用于背景道路网络的可视化

## 潜在问题
   - 用 OSM Shapefile 一次性合并了多个要素，将双向轨道、支线与主线都计算在内，可能与官方的“单向主线”的营业里程不一致
   - 在代码中采用简单的 1°=111 km 平面距离算法，会略微放大结果（特别是对经度方向）

In [ ]:
# 设置matplotlib使用widget后端
%matplotlib widget

import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd
import os
import numpy as np
from matplotlib.font_manager import FontProperties
from matplotlib.patches import Patch
from matplotlib.lines import Line2D
import shapely.geometry as sgeom
from shapely.geometry import Point

from shapely.ops import nearest_points  # 正确导入nearest_points函数
# 在不同版本的Shapely中，nearest_points函数可能位于不同的位置，可能应该使用from shapely.ops import nearest_points而不是shapely.geometry.ops。

# 设置中文字体为 Microsoft YaHei 或者 Noto Sans CJK JP
plt.rcParams['font.family'] = ['Microsoft YaHei']

# 定义文件路径配置函数
def configure_paths(root_path):
    """配置所有需要的文件路径
    
    参数:
        root_path (str): GIS数据文件根目录
        
    返回:
        dict: 包含所有文件路径的字典
    """
    paths = {
        'railways_file': os.path.join(root_path, 'gis_osm_railways_free_1.shp'),
        'transport_file': os.path.join(root_path, 'gis_osm_transport_a_free_1.shp'),
        'roads_file': os.path.join(root_path, 'gis_osm_roads_free_1.shp')
    }
    
    # 检查所有路径是否存在
    for key, path in paths.items():
        if not os.path.exists(path):
            print(f"警告: 文件不存在: {path}")
        else:
            print(f"文件存在: {path}")
    
    return paths

# 读取数据函数
def load_gis_data(file_path, verbose=True):
    """读取GIS数据
    
    参数:
        file_path (str): 数据文件路径
        verbose (bool): 是否显示详细信息
        
    返回:
        geopandas.GeoDataFrame: 读取的GIS数据
    """
    if not os.path.exists(file_path):
        print(f"错误: 文件不存在: {file_path}")
        return None
    
    if verbose:
        print(f"正在读取: {file_path}")
    
    data = gpd.read_file(file_path)
    
    if verbose:
        print(f"读取成功，共包含{len(data)}条记录")
    
    return data

# 计算两点间的距离
def calculate_distance(point1, point2):
    """
    使用GeoPandas和Shapely计算两点之间的地理距离（米）

    参数:
        point1 (list): 第一个点的坐标 [经度, 纬度]
        point2 (list): 第二个点的坐标 [经度, 纬度]

    返回:
        float: 两点间的距离（米）
    """

    # 创建GeoSeries，指定WGS84坐标系 (EPSG:4326)
    gdf = gpd.GeoSeries([Point(point1), Point(point2)], crs='EPSG:4326')
    
    # 将坐标投影到合适的投影坐标系（例如EPSG:3857，单位为米）
    gdf_proj = gdf.to_crs('EPSG:3857')
    
    # 计算投影后两个点之间的距离
    distance = gdf_proj[0].distance(gdf_proj[1])
    
    return distance

# 地铁线路分析函数
def analyze_metro_line(railways_data, line_name, verbose=True):
    """分析特定地铁线路
    
    参数:
        railways_data (geopandas.GeoDataFrame): 铁路数据
        line_name (str): 要分析的地铁线路名称
        verbose (bool): 是否显示详细信息
        
    返回:
        tuple: (线路数据, 坐标点数据, 分析统计信息)
    """
    if railways_data is None:
        print("错误: 铁路数据为空")
        return None, None, None
    
    # 筛选指定线路
    metro_line = railways_data[railways_data['name'] == line_name]
    
    if metro_line.empty:
        print(f"未找到名称为: {line_name} 的要素")
        
        # 尝试模糊匹配
        if verbose:
            line_number = ''.join(filter(str.isdigit, line_name))
            if line_number:
                contains_line = railways_data[railways_data['name'].str.contains(f'{line_number}号线', na=False)]
                if not contains_line.empty:
                    print(f"找到包含'{line_number}号线'的{len(contains_line)}条记录:")
                    print(contains_line['name'].unique())
        
        return None, None, None
    
    if verbose:
        print(f"找到 {len(metro_line)} 个匹配 {line_name} 的要素")
    
    # 从线路几何中提取所有坐标
    coordinates = []
    
    for idx, row in metro_line.iterrows():
        geom = row.geometry
        if geom.geom_type == 'LineString':
            # 添加此LineString中的所有点
            for i in range(len(geom.coords)):
                x, y = geom.coords[i]
                coordinates.append({
                    'segment_id': idx,
                    'point_id': i,
                    'longitude': x,
                    'latitude': y,
                    'osm_id': row['osm_id'],
                    'fclass': row['fclass'],
                    'bridge': row.get('bridge', 'N/A'),  # 获取是否为桥梁
                    'tunnel': row.get('tunnel', 'N/A')   # 获取是否为隧道
                })
        elif geom.geom_type == 'MultiLineString':
            # 处理MultiLineString几何（线路可能分段）
            for seg_idx, line in enumerate(geom.geoms):
                for i in range(len(line.coords)):
                    x, y = line.coords[i]
                    coordinates.append({
                        'segment_id': f"{idx}_{seg_idx}",
                        'point_id': i,
                        'longitude': x,
                        'latitude': y,
                        'osm_id': row['osm_id'],
                        'fclass': row['fclass'],
                        'bridge': row.get('bridge', 'N/A'),
                        'tunnel': row.get('tunnel', 'N/A')
                    })
    
    # 从坐标创建DataFrame
    coord_df = pd.DataFrame(coordinates)
    
    # 分析线路数据
    stats = calculate_line_statistics(coord_df)
    
    if verbose and stats:
        print(f"\n=== {line_name}数据分析 ===")
        for key, value in stats.items():
            print(f"{key}: {value}")
    
    return metro_line, coord_df, stats

# 线路统计信息计算函数
def calculate_line_statistics(coord_df):
    """计算地铁线路统计信息
    
    参数:
        coord_df (pandas.DataFrame): 线路坐标点数据
        
    返回:
        dict: 统计信息字典
    """
    if coord_df is None or coord_df.empty:
        return None
        
    # 计算线路总长度（使用GeoPandas和Shapely计算地理距离）
    total_length = 0
    segments = coord_df['segment_id'].unique()
    
    for segment in segments:
        segment_points = coord_df[coord_df['segment_id'] == segment]
        
        # 按point_id排序以确保正确的顺序
        segment_points = segment_points.sort_values('point_id')
        
        # 计算每段线路长度
        prev_point = None
        for _, point in segment_points.iterrows():
            if prev_point is not None:
                # 创建两个点的坐标 [经度, 纬度]
                point1 = [prev_point['longitude'], prev_point['latitude']]
                point2 = [point['longitude'], point['latitude']]
                
                # 使用calculate_distance函数计算精确的地理距离（米）
                dist_meters = calculate_distance(point1, point2)
                total_length += dist_meters
            
            prev_point = point
    
    # 计算桥梁和隧道比例
    bridge_segments = coord_df[coord_df['bridge'] == 'T']
    tunnel_segments = coord_df[coord_df['tunnel'] == 'T']
    
    # 准备统计信息
    stats = {
        "总点数": len(coord_df),
        "线段数": len(segments),
        "估计线路长度(公里)": round(total_length / 1000 / 2, 2),  # 双向长度取一半
        "坐标范围-经度": [coord_df['longitude'].min(), coord_df['longitude'].max()],
        "坐标范围-纬度": [coord_df['latitude'].min(), coord_df['latitude'].max()],
        "高架段点数": len(bridge_segments),
        "地下段点数": len(tunnel_segments)
    }
    
    return stats

# 站点分析函数
def analyze_station(transport_data, railways_data, roads_data, station_name, online_coords=None, line_name=None, verbose=True):
    """分析特定地铁站点，如果找多个同名站点，则基于距离选择最近的
    
    参数:
        transport_data (geopandas.GeoDataFrame): 运输站点数据
        railways_data (geopandas.GeoDataFrame): 铁路数据
        roads_data (geopandas.GeoDataFrame): 道路数据
        station_name (str): 站点名称
        online_coords (list): 可选的在线坐标 [经度, 纬度]
        line_name (str): 可选的线路名称，用于找最近点
        verbose (bool): 是否显示详细信息
        
    返回:
        dict: 站点分析结果
    """
    result = None
    
    if transport_data is not None:
        # 查找指定站点
        station_data = transport_data[transport_data['name'] == station_name]
        
        if not station_data.empty:
            if verbose:
                print(f"找到 {len(station_data)} 个{station_name}站点记录")
            
            # 如果找到多个同名站点且提供了在线坐标，基于距离选择最近的
            if len(station_data) > 1 and online_coords:
                # 为每个站点计算与在线坐标的距离
                distances = []
                for idx, station in station_data.iterrows():
                    station_coords = [station.geometry.centroid.x, station.geometry.centroid.y]
                    distance_meters = calculate_distance(station_coords, online_coords)
                    # distance_meters = distance * 111000  # 粗略转换为米
                    distances.append((idx, distance_meters))
                
                # 按距离排序并选择最近的站点
                closest_idx = sorted(distances, key=lambda x: x[1])[0][0]
                station = station_data.loc[closest_idx]
                
                if verbose:
                    print(f"发现多个同名站点，已选择距离在线坐标最近的站点（距离: {sorted(distances, key=lambda x: x[1])[0][1]:.2f}米）")
            else:
                # 只有一个站点或没有提供在线坐标，使用第一个匹配记录
                station = station_data.iloc[0]
            
            # 获取站点坐标（面状几何的中心点）
            station_coords = [station.geometry.centroid.x, station.geometry.centroid.y]
            
            # 创建结果字典
            result = {
                'station': station,
                'station_data': station_data,
                'station_coords': station_coords,
                'online_coords': online_coords,
                'from_nearest_point': False
            }
            
            # 如果提供了在线坐标，计算与在线坐标的距离
            if online_coords:
                distance_meters = calculate_distance(station_coords, online_coords)
                # distance_meters = distance * 111000  # 粗略转换为米
                result['distance_to_online'] = distance_meters
            
            if verbose:
                print(f"\n=== {station_name}站点信息 ===")
                print(f"坐标(中心点): 经度 {station_coords[0]:.7f}, 纬度 {station_coords[1]:.7f}")
                print(f"类型: {station['fclass']}")
                
                if online_coords:
                    print(f"离线shp文件坐标{station_name}站与openstreetmap网站记录坐标距离: {distance_meters:.2f} 米")
                
                # 显示所有可用的属性
                print("\n完整属性信息:")
                for column in station_data.columns:
                    if column != 'geometry':
                        value = station[column]
                        if pd.notna(value):  # 只显示非空值
                            print(f"{column}: {value}")
        else:
            if verbose:
                print(f"在运输站点数据中未找到{station_name}站点")
    
    # 如果没有找到站点但提供了坐标和线路名称，尝试找最近点
    if result is None and online_coords and line_name and railways_data is not None:
        if verbose:
            print(f"尝试在{line_name}上找到最接近{online_coords}的点...")
        
        nearest_point = find_nearest_point_on_line(railways_data, line_name, online_coords)
        
        if nearest_point:
            # 创建一个虚拟站点对象
            from shapely.geometry import Point
            virtual_station = gpd.GeoDataFrame(
                {
                    'osm_id': [nearest_point['properties']['osm_id']],
                    'fclass': [nearest_point['properties']['fclass']],
                    'name': [station_name],
                    'bridge': [nearest_point['properties']['bridge']],
                    'tunnel': [nearest_point['properties']['tunnel']]
                },
                geometry=[Point(nearest_point['coords'][0], nearest_point['coords'][1])]
            )
            
            result = {
                'station': virtual_station.iloc[0],
                'station_data': virtual_station,
                'station_coords': nearest_point['coords'],
                'online_coords': online_coords,
                'from_nearest_point': True,
                'distance_to_online': nearest_point['distance']
            }
            
            if verbose:
                print(f"\n=== 已创建虚拟{station_name}站点 ===")
                print(f"坐标: 经度 {nearest_point['coords'][0]:.7f}, 纬度 {nearest_point['coords'][1]:.7f}")
                print(f"与提供的在线坐标距离: {nearest_point['distance']:.2f} 米")
    
    return result

# 在线路上找最近点
def find_nearest_point_on_line(railways_data, line_name, online_coords):
    """找到地铁线路上距离给定坐标最近的点
    
    参数:
        railways_data (geopandas.GeoDataFrame): 铁路数据
        line_name (str): 线路名称
        online_coords (list): 在线坐标 [经度, 纬度]
        
    返回:
        dict: 包含最近点信息的字典
    """
    if railways_data is None:
        print("错误: 铁路数据为空")
        return None
    
    # 筛选指定线路
    metro_line = railways_data[railways_data['name'] == line_name]
    if metro_line.empty:
        print(f"未找到名称为: {line_name} 的线路")
        return None
    
    # 创建坐标点
    point = sgeom.Point(online_coords[0], online_coords[1])
    
    # 初始化最近距离和最近点
    min_distance = float('inf')
    nearest_point_info = None
    
    # 遍历线路的所有段
    for idx, row in metro_line.iterrows():
        geom = row.geometry
        
        # 找到此几何对象上的最近点
        nearest_geom_point = nearest_points(point, geom)[1]  # 使用正确导入的函数
        
        # 计算距离
        distance = point.distance(nearest_geom_point)
        
        # 如果这是最近的点，更新信息
        if distance < min_distance:
            min_distance = distance
            
            # 将距离转换为米（粗略估计）
            distance_meters = distance * 111000
            
            nearest_point_info = {
                'segment_id': idx,
                'geometry': nearest_geom_point,
                'coords': [nearest_geom_point.x, nearest_geom_point.y],
                'distance': distance_meters,
                'properties': {
                    'osm_id': row['osm_id'],
                    'fclass': row['fclass'],
                    'bridge': row.get('bridge', 'N/A'),
                    'tunnel': row.get('tunnel', 'N/A')
                }
            }
    
    if nearest_point_info:
        print(f"找到线路上距离在线坐标最近的点，距离约 {nearest_point_info['distance']:.2f} 米")
        print(f"坐标: 经度 {nearest_point_info['coords'][0]:.7f}, 纬度 {nearest_point_info['coords'][1]:.7f}")
        print(f"类型: {nearest_point_info['properties']['fclass']}")
        print(f"是否高架: {'是' if nearest_point_info['properties']['bridge'] == 'T' else '否'}")
        print(f"是否地下: {'是' if nearest_point_info['properties']['tunnel'] == 'T' else '否'}")
    
    return nearest_point_info

# 创建分类线路段函数
def create_categorized_line_segments(metro_line):
    """将地铁线路分为桥梁、隧道和普通路段
    
    参数:
        metro_line (geopandas.GeoDataFrame): 地铁线路数据
        
    返回:
        tuple: (桥梁段, 隧道段, 普通路段) 的GeoDataFrames
    """
    if metro_line is None or metro_line.empty:
        return None, None, None
    
    # 创建桥梁、隧道和普通路段的筛选
    bridge_segments = metro_line[metro_line['bridge'] == 'T'].copy()
    tunnel_segments = metro_line[metro_line['tunnel'] == 'T'].copy()
    
    # 普通路段 (既不是桥梁也不是隧道)
    normal_segments = metro_line[
        (metro_line['bridge'] != 'T') & 
        (metro_line['tunnel'] != 'T')
    ].copy()
    
    return bridge_segments, tunnel_segments, normal_segments

# 用于查找线路附近的所有站点
def find_stations_near_line(railways_data, transport_data, line_name, max_distance=0.005):
    """查找地铁线路附近的所有站点
    
    参数:
        railways_data (geopandas.GeoDataFrame): 铁路数据
        transport_data (geopandas.GeoDataFrame): 运输站点数据
        line_name (str): 线路名称
        max_distance (float): 最大距离（度）
        
    返回:
        pandas.DataFrame: 附近站点列表
    """
    if railways_data is None or transport_data is None:
        print("错误: 数据为空")
        return None
    
    # 筛选指定线路
    metro_line = railways_data[railways_data['name'] == line_name]
    if metro_line.empty:
        print(f"未找到名称为: {line_name} 的线路")
        return None
    
    # 创建线路几何对象的并集 - 使用推荐的union_all()方法
    line_geometry = metro_line.geometry.union_all()
    
    # 创建缓冲区 (约500米)
    buffer = line_geometry.buffer(max_distance)
    
    # 查找缓冲区内的站点
    stations_in_buffer = transport_data[transport_data.geometry.intersects(buffer)]
    
    # 全面筛选所有站台相关设施
    metro_stations = stations_in_buffer[
        # 基于设施类型筛选所有可能的站点类型
        (stations_in_buffer['fclass'].str.contains('subway|railway|station|platform|stop|halt|terminal', na=False)) |
        
        # 基于名称筛选可能的站点
        (stations_in_buffer['name'].str.contains('站$|站台|车站|地铁|火车|高铁|动车|客运|枢纽', na=False))
    ]

    # 原有的后备方案保持不变
    if metro_stations.empty:
        print("在线路附近未找到任何站点相关设施，显示所有交通设施:")
        metro_stations = stations_in_buffer         
    
    # 如果没有匹配到站点，尝试显示所有交通设施
    if metro_stations.empty:
        print("在线路附近未找到明确标记为地铁站的记录，显示所有交通设施:")
        metro_stations = stations_in_buffer
    
    # 输出找到的站点
    if not metro_stations.empty:
        print(f"\n=== 在{line_name}附近找到{len(metro_stations)}个交通设施 ===")
        
        # 将几何体转换为投影坐标系计算中心点 (UTM Zone 51N适合上海地区)
        # 注意：保留原始数据，仅在计算中心点时使用投影坐标系
        metro_stations_projected = metro_stations.to_crs("EPSG:32651")
        
        result_df = pd.DataFrame({
            'name': metro_stations['name'],
            'fclass': metro_stations['fclass'],
            'longitude': metro_stations.geometry.centroid.x,  # 仍然使用原始经纬度显示
            'latitude': metro_stations.geometry.centroid.y    # 仍然使用原始经纬度显示
        })
        
        # 按名称排序
        result_df = result_df.sort_values('name')
        
        return result_df
    else:
        print(f"在{line_name}附近未找到任何交通设施")
        return None

# 可视化函数 - 站点对比图
def visualize_station_comparison(station_result, railways_data, roads_data, line_name=None):
    """创建站点对比可视化图
    
    参数:
        station_result (dict): 站点分析结果
        railways_data (geopandas.GeoDataFrame): 铁路数据
        roads_data (geopandas.GeoDataFrame): 道路数据
        line_name (str): 可选的线路名称，用于高亮显示
        
    返回:
        matplotlib.figure.Figure: 可视化图表
    """
    if station_result is None:
        print("错误: 站点分析结果为空")
        return None
    
    station = station_result['station']
    station_coords = station_result['station_coords']
    online_coords = station_result['online_coords']
    station_name = station['name']
    from_nearest_point = station_result.get('from_nearest_point', False)
    
    # 创建可视化图
    fig, ax = plt.subplots(figsize=(14, 10))
    
    # 添加道路作为背景
    if roads_data is not None:
        roads_data.plot(ax=ax, color='lightgray', linewidth=0.5, alpha=0.5)
    
    # 添加地铁线路
    if railways_data is not None:
        subway_lines = railways_data[railways_data['fclass'] == 'subway']
        subway_lines.plot(ax=ax, color='gray', linewidth=1.0, alpha=0.5)
        
        # 特别高亮显示指定线路，根据桥梁和隧道属性区分颜色
        if line_name:
            specific_line = railways_data[railways_data['name'] == line_name]
            if not specific_line.empty:
                # 分类线路段
                bridge_segments, tunnel_segments, normal_segments = create_categorized_line_segments(specific_line)
                
                # 绘制普通路段 (蓝色)
                if normal_segments is not None and not normal_segments.empty:
                    normal_segments.plot(ax=ax, color='blue', linewidth=2.5, label='地面段')
                
                # 绘制桥梁段 (红色)
                if bridge_segments is not None and not bridge_segments.empty:
                    bridge_segments.plot(ax=ax, color='red', linewidth=2.5, label='高架桥段')
                
                # 绘制隧道段 (绿色)
                if tunnel_segments is not None and not tunnel_segments.empty:
                    tunnel_segments.plot(ax=ax, color='green', linewidth=2.5, label='地下隧道段')
    
    # 根据是否为虚拟站点处理不同的绘图方式
    if from_nearest_point:
        # 绘制站点点
        ax.scatter(station_coords[0], station_coords[1], 
                   color='orange', edgecolor='black', s=100, zorder=5, 
                   label=f'创建的{station_name}站点位置')
    else:
        # 绘制站点几何形状
        station_gdf = gpd.GeoDataFrame(geometry=[station.geometry])
        station_gdf.plot(ax=ax, color='yellow', edgecolor='black', alpha=0.7)
        
        # 绘制站点中心点
        ax.scatter(station_coords[0], station_coords[1], 
                   color='orange', edgecolor='black', s=100, zorder=5, 
                   label=f'站点中心点 (Shapefile离线数据)')
    
    # 如果提供了在线坐标，绘制对比信息
    if online_coords:
        # 绘制在线坐标
        ax.scatter(online_coords[0], online_coords[1], 
                   color='purple', edgecolor='black', s=100, zorder=5, 
                   label=f'在线坐标 ({online_coords[1]}, {online_coords[0]})')
        
        # 绘制连接线，显示两点之间的差异
        ax.plot([station_coords[0], online_coords[0]], 
                [station_coords[1], online_coords[1]], 
                'k--', linewidth=1.5, zorder=4)
        
        # 调整视图范围 - 聚焦于站点附近区域
        margin = 0.005  # 边距（度）
        avg_lon = (station_coords[0] + online_coords[0]) / 2
        avg_lat = (station_coords[1] + online_coords[1]) / 2
    else:
        # 如果没有在线坐标，以站点坐标为中心
        margin = 0.005  # 边距（度）
        avg_lon = station_coords[0]
        avg_lat = station_coords[1]
        
        # 添加文本标注
        ax.text(station_coords[0], station_coords[1] + 0.001, 
                station_name, 
                ha='center', va='bottom', fontsize=10)
    
    # 设置图表标题和标签
    if from_nearest_point:
        plt.title(f'{station_name}站点位置（根据在线坐标创建）')
    else:
        plt.title(f'{station_name}站点对比图')
    
    plt.xlabel('经度')
    plt.ylabel('纬度')
    
    # 调整视图范围
    plt.xlim(avg_lon - margin, avg_lon + margin)
    plt.ylim(avg_lat - margin, avg_lat + margin)
    
    # 添加图例
    handles, labels = ax.get_legend_handles_labels()
    
    # 手动添加站点面状几何的图例（如果不是虚拟站点）
    if not from_nearest_point:
        handles.append(Patch(facecolor='yellow', alpha=0.7, edgecolor='black'))
        labels.append('站点区域 (Shapefile离线数据)')
    
    plt.legend(handles, labels, loc='upper right')
    
    # 添加比例尺和指北箭头
    # 比例尺 (0.001度 ≈ 111米)
    scale_bar_length = 0.001
    x_scale = avg_lon - margin * 0.8
    y_scale = avg_lat - margin * 0.8
    plt.plot([x_scale, x_scale + scale_bar_length], [y_scale, y_scale], 'k-', lw=2)
    plt.text(x_scale + scale_bar_length/2, y_scale - margin*0.1, 
             f"{int(scale_bar_length*111000)}米", ha='center')
    
    # 指北箭头
    arrow_props = dict(arrowstyle='->', linewidth=2, color='black')
    ax.annotate('N', xy=(0.05, 0.95), xycoords='axes fraction',
               xytext=(0.05, 0.85), textcoords='axes fraction',
               arrowprops=arrow_props, ha='center', va='center', fontsize=14)
    
    return fig

# 可视化函数 - 整体线路图
def visualize_network_overview(station_result, railways_data, line_name=None):
    """创建整体线路图，显示站点在地铁网络中的位置
    
    参数:
        station_result (dict): 站点分析结果
        railways_data (geopandas.GeoDataFrame): 铁路数据
        line_name (str): 可选的线路名称，用于高亮显示
        
    返回:
        matplotlib.figure.Figure: 可视化图表
    """
    if station_result is None:
        print("错误: 站点分析结果为空")
        return None
    
    station_coords = station_result['station_coords']
    station_name = station_result['station']['name']
    from_nearest_point = station_result.get('from_nearest_point', False)
    
    # 创建图表
    fig2, ax2 = plt.subplots(figsize=(14, 10))
    
    # 添加地铁线路
    if railways_data is not None:
        subway_lines = railways_data[railways_data['fclass'] == 'subway']
        subway_lines.plot(ax=ax2, color='lightgray', linewidth=1.0, alpha=0.5)
        
        # 高亮显示指定线路，根据桥梁和隧道属性区分颜色
        if line_name:
            specific_line = railways_data[railways_data['name'] == line_name]
            if not specific_line.empty:
                # 分类线路段
                bridge_segments, tunnel_segments, normal_segments = create_categorized_line_segments(specific_line)
                
                # 绘制普通路段 (蓝色)
                if normal_segments is not None and not normal_segments.empty:
                    normal_segments.plot(ax=ax2, color='blue', linewidth=2.0, label='地面段')
                
                # 绘制桥梁段 (红色)
                if bridge_segments is not None and not bridge_segments.empty:
                    bridge_segments.plot(ax=ax2, color='red', linewidth=2.0, label='高架桥段')
                
                # 绘制隧道段 (绿色)
                if tunnel_segments is not None and not tunnel_segments.empty:
                    tunnel_segments.plot(ax=ax2, color='green', linewidth=2.0, label='地下隧道段')
    
    # 绘制站点位置
    if from_nearest_point:
        ax2.scatter(station_coords[0], station_coords[1], 
                   color='orange', edgecolor='black', s=100, zorder=5, 
                   label=f'{station_name}（根据在线坐标创建）')
    else:
        ax2.scatter(station_coords[0], station_coords[1], 
                   color='orange', edgecolor='black', s=100, zorder=5, 
                   label=station_name)
    
    if from_nearest_point:
        plt.title(f'地铁网络中的{station_name}位置（根据在线坐标创建）')
    else:
        plt.title(f'地铁网络中的{station_name}位置')
    
    plt.xlabel('经度')
    plt.ylabel('纬度')
    plt.legend()
    
    return fig2

# 可视化函数 - 仅显示线路结构图
def visualize_line_structure(railways_data, line_name, coord_df=None):
    """创建线路结构图，显示桥梁、隧道和普通路段
    
    参数:
        railways_data (geopandas.GeoDataFrame): 铁路数据
        line_name (str): 线路名称
        coord_df (pandas.DataFrame): 可选的线路坐标数据，用于添加更详细的分析
        
    返回:
        matplotlib.figure.Figure: 可视化图表
    """
    if railways_data is None:
        print("错误: 铁路数据为空")
        return None
    
    # 筛选指定线路
    specific_line = railways_data[railways_data['name'] == line_name]
    if specific_line.empty:
        print(f"未找到名称为: {line_name} 的线路")
        return None
    
    # 创建图表
    fig, ax = plt.subplots(figsize=(14, 8))
    
    # 分类线路段
    bridge_segments, tunnel_segments, normal_segments = create_categorized_line_segments(specific_line)
    
    # 绘制普通路段 (蓝色)
    if normal_segments is not None and not normal_segments.empty:
        normal_segments.plot(ax=ax, color='blue', linewidth=2.5, label='地面段')
        print(f"地面段数量: {len(normal_segments)}")
    
    # 绘制桥梁段 (红色)
    if bridge_segments is not None and not bridge_segments.empty:
        bridge_segments.plot(ax=ax, color='red', linewidth=2.5, label='高架桥段')
        print(f"高架桥段数量: {len(bridge_segments)}")
    
    # 绘制隧道段 (绿色)
    if tunnel_segments is not None and not tunnel_segments.empty:
        tunnel_segments.plot(ax=ax, color='green', linewidth=2.5, label='地下隧道段')
        print(f"地下隧道段数量: {len(tunnel_segments)}")
    
    # 计算统计信息
    if coord_df is not None and not coord_df.empty:
        bridge_points = coord_df[coord_df['bridge'] == 'T']
        tunnel_points = coord_df[coord_df['tunnel'] == 'T']
        normal_points = coord_df[(coord_df['bridge'] != 'T') & (coord_df['tunnel'] != 'T')]
        
        bridge_pct = len(bridge_points) / len(coord_df) * 100 if len(coord_df) > 0 else 0
        tunnel_pct = len(tunnel_points) / len(coord_df) * 100 if len(coord_df) > 0 else 0
        normal_pct = len(normal_points) / len(coord_df) * 100 if len(coord_df) > 0 else 0
        
        # 添加统计信息文本框
        textstr = '\n'.join([
            f"线路构成:",
            f"地面段: {normal_pct:.1f}%",
            f"高架桥段: {bridge_pct:.1f}%",
            f"地下隧道段: {tunnel_pct:.1f}%"
        ])
        
        props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)
        ax.text(0.05, 0.95, textstr, transform=ax.transAxes, fontsize=10,
                verticalalignment='top', bbox=props)
    
    plt.title(f'{line_name}路段类型分布')
    plt.xlabel('经度')
    plt.ylabel('纬度')
    
    # 调整视图以适应线路
    bounds = specific_line.total_bounds
    margin = 0.01  # 边距（度）
    plt.xlim(bounds[0] - margin, bounds[2] + margin)
    plt.ylim(bounds[1] - margin, bounds[3] + margin)
    
    # 添加图例
    plt.legend(loc='upper right')
    
    return fig

# 保存分析结果函数
def save_analysis_results(root_path, city, line_name, line_coord_df, station_name=None, save_visualization=False, figures=None):
    """保存分析结果到文件
    
    参数:
        root_path (str): 根目录路径
        line_name (str): 线路名称
        line_coord_df (pandas.DataFrame): 线路坐标数据
        station_name (str): 可选的站点名称
        save_visualization (bool): 是否保存可视化图表
        figures (dict): 包含图表对象的字典
        
    返回:
        dict: 保存的文件路径
    """
    saved_files = {}
    
    # 提取线路编号
    line_number = ''.join(filter(str.isdigit, line_name))
    
    # 保存线路坐标
    if line_coord_df is not None and not line_coord_df.empty:
        line_csv_path = os.path.join(root_path, f'{city}_metro_line{line_number}_coordinates.csv')
        line_coord_df.to_csv(line_csv_path, index=False, encoding='utf-8')
        saved_files['line_coordinates'] = line_csv_path
        print(f"已将 {len(line_coord_df)} 个坐标点保存到 {line_csv_path}")
    
    # 保存可视化图表
    if save_visualization and figures:
        if 'line_structure' in figures:
            structure_path = os.path.join(root_path, f'{city}_metro_line{line_number}_structure.png')
            figures['line_structure'].savefig(structure_path, dpi=300)
            saved_files['line_structure'] = structure_path
            print(f"线路结构图已保存到 {structure_path}")
            
        if 'station_comparison' in figures and station_name:
            comparison_path = os.path.join(root_path, f'{station_name}_detailed.png')
            figures['station_comparison'].savefig(comparison_path, dpi=300)
            saved_files['station_comparison'] = comparison_path
            print(f"站点对比图已保存到 {comparison_path}")
        
        if 'network_overview' in figures and station_name:
            overview_path = os.path.join(root_path, f'{station_name}_overview.png')
            figures['network_overview'].savefig(overview_path, dpi=300)
            saved_files['network_overview'] = overview_path
            print(f"整体线路图已保存到 {overview_path}")
    
    return saved_files

# 主函数
def analyze_city_metro(root_path, city, line_name, station_name=None, online_coords=None, save_results=False):
    """分析地铁线路和站点
    
    参数:
        root_path (str): GIS数据文件根目录
        line_name (str): 要分析的地铁线路名称
        station_name (str): 可选的站点名称
        online_coords (list): 可选的站点在线坐标 [经度, 纬度]
        save_results (bool): 是否保存分析结果
        
    返回:
        dict: 分析结果
    """
    # 配置路径
    paths = configure_paths(root_path)
    
    # 读取数据
    railways_data = load_gis_data(paths['railways_file'])
    transport_data = load_gis_data(paths['transport_file']) if station_name else None
    roads_data = load_gis_data(paths['roads_file']) if station_name else None
    
    # 分析线路
    metro_line, line_coord_df, line_stats = analyze_metro_line(railways_data, line_name)
    
    # 先初始化结果字典
    results = {
        'line_name': line_name,
        'line_data': metro_line,
        'line_coordinates': line_coord_df,
        'line_statistics': line_stats
    }
    
    # 然后查找线路附近的站点并添加到结果中
    nearby_stations = find_stations_near_line(railways_data, transport_data, line_name)
    if nearby_stations is not None:
        print("\n可用的站点名称:")
        for _, row in nearby_stations.iterrows():
            print(f"{row['name']} - 类型: {row['fclass']} - 坐标: [{row['longitude']:.7f}, {row['latitude']:.7f}]")
        results['nearby_stations'] = nearby_stations
    
    figures = {}
    
    # 创建线路结构可视化
    line_structure_fig = visualize_line_structure(railways_data, line_name, line_coord_df)
    if line_structure_fig:
        figures['line_structure'] = line_structure_fig
    
    # 如果指定了站点名称，分析站点
    if station_name:
        # 修改此处，传入line_name参数
        station_result = analyze_station(transport_data, railways_data, roads_data, station_name, online_coords, line_name)
        results['station_result'] = station_result
        
        if station_result:
            # 创建站点对比图
            fig1 = visualize_station_comparison(station_result, railways_data, roads_data, line_name)
            figures['station_comparison'] = fig1
            
            # 创建整体线路图
            fig2 = visualize_network_overview(station_result, railways_data, line_name)
            figures['network_overview'] = fig2
    
    # 保存结果
    if save_results:
        saved_files = save_analysis_results(root_path, city, line_name, line_coord_df, station_name, True, figures)
        results['saved_files'] = saved_files
    
    return results, figures

# 示例用法
if __name__ == "__main__":

    # 配置文件，线路名称可以从 ArcGIS Pro 之中读取
    # [经度, 纬度]；注意与 openstreetmap 的“位置”项正好相反

    root_path = r'E:\ResearchDocuments\ROS2WithSPCK\GIS_Track\路网GIS数据集\shanghai-latest-free.shp'
    city = 'Shanghai'
    line_name = '沪苏湖高速铁路'
    station_name = '春申站'
    online_coords = [121.3495373,  31.0801540] # [经度, 纬度] 
    
    # 执行分析
    results, figures = analyze_city_metro(root_path, city, line_name, station_name, online_coords, save_results=True)


### 需要重点关注的上海线路与站点

In [ ]:
"""

1) SH17 - 国家会展中心
    root_path = r'E:\ResearchDocuments\ROS2WithSPCK\GIS_Track\路网GIS数据集\shanghai-latest-free.shp'
    city = 'Shanghai'
    line_name = '上海地铁17号线'
    station_name = '国家会展中心'
    online_coords = [121.2888239, 31.1936744]  # [经度, 纬度] 靠近17号线的站点


2) SH17 - 虹桥火车站
    root_path = r'E:\ResearchDocuments\ROS2WithSPCK\GIS_Track\路网GIS数据集\shanghai-latest-free.shp'
    city = 'Shanghai'
    line_name = '上海地铁17号线'
    station_name = '上海虹桥客运站'
    online_coords = [121.3143014, 31.1960757]  # [经度, 纬度] 


3) 市域铁机场联络线 - 虹桥2号航站楼
    root_path = r'E:\ResearchDocuments\ROS2WithSPCK\GIS_Track\路网GIS数据集\shanghai-latest-free.shp'
    city = 'Shanghai'
    line_name = '市域铁机场联络线'
    station_name = '虹桥2号航站楼'
    online_coords = [121.3198748,31.1961340]  # [经度, 纬度] 


4) 市域铁机场联络线 - 中春路
    root_path = r'E:\ResearchDocuments\ROS2WithSPCK\GIS_Track\路网GIS数据集\shanghai-latest-free.shp'
    city = 'Shanghai'
    line_name = '市域铁机场联络线'
    station_name = '中春路'
    online_coords = [121.3299336,  31.1519780]  # [经度, 纬度] 在线坐标更加准确

5) 沪苏湖高速铁路
    root_path = r'E:\ResearchDocuments\ROS2WithSPCK\GIS_Track\路网GIS数据集\shanghai-latest-free.shp'
    city = 'Shanghai'
    line_name = '沪苏湖高速铁路'
    station_name = '春申站'
    online_coords = [121.3495373,  31.0801540] # [经度, 纬度] 

"""

### 其他用于测试的站点

In [ ]:
"""

3) GZ18
    root_path = r'E:\ResearchDocuments\ROS2WithSPCK\GIS_Track\路网GIS数据集\guangdong-latest-free.shp'
    city = 'Guangzhou'
    line_name = '广州地铁18号线'
    station_name = '万顷沙'
    online_coords = [113.5519154, 22.7014202]  # [经度, 纬度]

1) SH11
    root_path = r'E:\ResearchDocuments\ROS2WithSPCK\GIS_Track\路网GIS数据集\shanghai-latest-free.shp'
    city = 'Shanghai'
    line_name = '上海地铁11号线'
    station_name = '上海汽车城'
    online_coords = [121.1768832,  31.287150]  # [经度, 纬度]

4) GZ18
    root_path = r'E:\ResearchDocuments\ROS2WithSPCK\GIS_Track\路网GIS数据集\guangdong-latest-free.shp'
    city = 'Guangzhou'
    line_name = '广州地铁18号线'
    station_name = '龙潭站'
    online_coords = [113.3356412, 23.0827287]  # [经度, 纬度]
    
5) GQCJ
    root_path = r'E:\ResearchDocuments\ROS2WithSPCK\GIS_Track\路网GIS数据集\guangdong-latest-free.shp'
    city = 'Guangzhou'
    line_name = '广清城际线'
    station_name = '花都站'
    online_coords = [113.1973668,  23.3798674]  # [经度, 纬度]

6) DXJCLLX
    root_path = r'E:\ResearchDocuments\ROS2WithSPCK\GIS_Track\路网GIS数据集\beijing-latest-free.shp'
    city = 'Beijing'
    line_name = '大兴机场线'
    station_name = '北京大兴国际机场'
    online_coords = [116.4095525,  39.5129055]  # [经度, 纬度] 

7) DXJCLLX     
    root_path = r'E:\ResearchDocuments\ROS2WithSPCK\GIS_Track\路网GIS数据集\beijing-latest-free.shp'
    city = 'Beijing'
    line_name = '大兴机场线'
    station_name = '草桥'
    online_coords = [116.3424361,  39.8461391]  # [经度, 纬度]
       
8) CGTL
    root_path = r'E:\ResearchDocuments\ROS2WithSPCK\GIS_Track\路网GIS数据集\sichuan-latest-free.shp'
    city = 'Sichuan'
    line_name = '成灌铁路'
    station_name = '犀浦'
    online_coords = [103.9706133, 30.7597164]  # [经度, 纬度]
       
9) CD2
    root_path = r'E:\ResearchDocuments\ROS2WithSPCK\GIS_Track\路网GIS数据集\sichuan-latest-free.shp'
    city = 'Sichuan'
    line_name = '成都地铁2号线'
    station_name = '犀浦'
    online_coords = [103.9706133, 30.7597164]  # [经度, 纬度]

10) XA1
    root_path = r'E:\ResearchDocuments\ROS2WithSPCK\GIS_Track\路网GIS数据集\shaanxi-latest-free.shp'
    city = 'Shaanxi'
    line_name = '西户线' 
    station_name = '阿房宫南'
    online_coords = [108.8208194, 34.2540384]  # [经度, 纬度]

11) SH17
    root_path = r'E:\ResearchDocuments\ROS2WithSPCK\GIS_Track\路网GIS数据集\shanghai-latest-free.shp'
    city = 'Shanghai'
    line_name = '上海地铁17号线'
    station_name = '蟠龙路'
    online_coords = [121.2742138, 31.1884239]  # [经度, 纬度]

"""